# ME 617 HW 3

**Author: Cameron Irmas**

_Note: This source code was implemented in Julia. This notebook requires a Julia 1.7.x kernel as well as an environment that includes the necessary dependencies. See the [repo](https://github.com/camirmas/DesignAutomation) for full implementations, testing, and dependency information. All relevant code has been copied into this notebook, so no importing of individual modules is necessary._

In [57]:
using LinearAlgebra
using Optim

In [58]:
function levy3(x)
    n = length(x)

    y(x_i) = 1 + (x_i-1)/4

    term1 = sin(π*y(x[1]))^2
    term3 = y(x[n]-1)^2 * (1+sin(2π*y(x[n]))^2)
    
    sum = 0
    for i=1:n-1
        new = (y(x[i])-1)^2*(1+10sin(π*y(x[i])+1)^2)
        sum += new
    end

    return term1 + sum + term3
end

levy3 (generic function with 1 method)

## 1 Exhaustive Search

In [59]:
"""
Performs an exhaustive search for a function using a discretized search space.

The total number of function calls will be d^n, where d is the number of
dimensions, and n is the length of the discretized search space.

    Args:
      fn (Function): Objective function
      x_range (Array): Range of x values to explore
      dimensions (Int64): Dimensionality of search space
      spacing (Float64, optional): Discretization
      verbose (Bool, optional): Print results

    Returns:
      Tuple: (x, fx, f_calls) where x is the minimizer, fx is the minimum, and
      f_calls is the number of function calls made.
"""
function exhaustive_search(fn::Function, x_range::Array, dimensions::Int64;
                           spacing=.1, verbose=true)
    f_calls = 1

    i = 1
    k = [1 for _ in 1:dimensions]
    min_x, max_x = x_range
    x_values = collect(min_x:spacing:max_x)
    n_x_values = length(x_values)
    x = [x_values[begin] for _ in 1:dimensions]
    x_star = copy(x)
    f_star = fn(x_star)
    n = length(x)

    while i <= n
        k[i] += 1

        if k[i] > n_x_values
            x[i] = min_x
            k[i] = 1
            i += 1
            continue
        end

        x[i] = x_values[k[i]]

        fnew = fn(x)
        f_calls += 1

        if verbose && (f_calls % 1000000 == 0)
            println("Iterations: $(f_calls)\nx*: $(x_star)\nf*: $(f_star)\n")
        end

        if fnew < f_star
            f_star = fnew
            x_star = copy(x)
        end

        i = 1
    end

    if verbose
        println("Iterations: $(f_calls)\nx*: $(x_star)\nf*: $(f_star)\n")
    end

    return x_star, f_star, f_calls
end

exhaustive_search

In [60]:
(x, fx, f_calls) = exhaustive_search(levy3, [-2, 6], 4; verbose=true)

Iterations: 1000000
x*: [1.0, 1.0, 1.0, -2.0]
f*: 1.4997597826618576e-32

Iterations: 2000000
x*: [1.0, 1.0, 1.0, -2.0]
f*: 1.4997597826618576e-32

Iterations: 3000000
x*: [1.0, 1.0, 1.0, -2.0]
f*: 1.4997597826618576e-32

Iterations: 4000000
x*: [1.0, 1.0, 1.0, -2.0]
f*: 1.4997597826618576e-32

Iterations: 5000000
x*: [1.0, 1.0, 1.0, -2.0]
f*: 1.4997597826618576e-32

Iterations: 6000000
x*: [1.0, 1.0, 1.0, -2.0]
f*: 1.4997597826618576e-32

Iterations: 7000000
x*: [1.0, 1.0, 1.0, -2.0]
f*: 1.4997597826618576e-32

Iterations: 8000000
x*: [1.0, 1.0, 1.0, -2.0]
f*: 1.4997597826618576e-32

Iterations: 9000000
x*: [1.0, 1.0, 1.0, -2.0]
f*: 1.4997597826618576e-32

Iterations: 10000000
x*: [1.0, 1.0, 1.0, -2.0]
f*: 1.4997597826618576e-32

Iterations: 11000000
x*: [1.0, 1.0, 1.0, -2.0]
f*: 1.4997597826618576e-32

Iterations: 12000000
x*: [1.0, 1.0, 1.0, -2.0]
f*: 1.4997597826618576e-32

Iterations: 13000000
x*: [1.0, 1.0, 1.0, -2.0]
f*: 1.4997597826618576e-32

Iterations: 14000000
x*: [1.0, 1.0

([1.0, 1.0, 1.0, -2.0], 1.4997597826618576e-32, 43046721)

## 2 Random Hill Climbing

In [61]:
"""
Performs a random hill climb solution for a discretized search space.

    Args:
      fn (Function): Objective function
      x0 (Array{Any}): Starting point
      h (Float64, optional): Discretization
      max_failed (Int64, optional): Max number of failed moves before exiting.
        Defaults to the dimensionality of the design space.
      verbose (Bool, optional): Optionally print results

    Returns:
      tuple: (x, fx, f_calls) where x is the minimizer, fx is the minimum, and
      f_calls is the number of function calls made.
"""
function random_hill(fn::Function, x0::Array{Float64};
                     h=.1, max_failed=nothing, verbose=false)
    x = copy(x0)
    fx = fn(x)
    n = length(x)
    failed = 0
    f_calls = 1

    # use an I matrix to randomly determine direction
    e_hat = Matrix(1I, n, n)

    if isnothing(max_failed)
        max_failed = n
    end

    while failed < max_failed
        d = e_hat[:, rand(1:n)] # randomly choose basis vec
        d *= rand([-1, 1]) # randomly choose pos/neg
        
        xnew = x + h*d
        fnew = fn(xnew)
        f_calls += 1

        if fnew < fx
            fx = fnew
            x = xnew
            failed = 0
        else
            failed += 1
        end
    end

    if verbose
        println("Iterations: $(f_calls)\nx*: $(x)\nf*: $(fx)\n")
    end

    return x, fx, f_calls
end

random_hill

In [62]:
x0 = [6.0, -2.0, -2.0, 6.0]
random_hill(levy3, x0; max_failed=100, verbose=true)
random_hill(levy3, x0; h=.01, max_failed=100, verbose=true)

Iterations: 279
x*: [3.800000000000007, -0.09999999999999937, -0.09999999999999937, 4.900000000000004]
f*: 4.388225844856102

Iterations: 1989
x*: [3.790000000000047, -0.08999999999999836, -0.08999999999999836, 4.940000000000023]
f*: 4.376401911442446



([3.790000000000047, -0.08999999999999836, -0.08999999999999836, 4.940000000000023], 4.376401911442446, 1989)

In [63]:
x0 = [4.2, -1.2, 2.4, -3.8]
random_hill(levy3, x0; max_failed=100, verbose=true)
random_hill(levy3, x0; h=.01, max_failed=100, verbose=true)

Iterations: 249
x*: [3.8000000000000007, -0.10000000000000003, 0.9999999999999988, -1.9999999999999982]
f*: 1.25024294018724

Iterations: 1571
x*: [3.790000000000009, -0.08999999999999903, 1.0000000000000075, -2.000000000000038]
f*: 1.2499870520527887



([3.790000000000009, -0.08999999999999903, 1.0000000000000075, -2.000000000000038], 1.2499870520527887, 1571)

In [64]:
x0 = [0.0, 0.0, 0.0, 3.2]
random_hill(levy3, x0; max_failed=100, verbose=true)
random_hill(levy3, x0; h=.01, max_failed=100, verbose=true)

Iterations: 178
x*: [0.9999999999999999, -0.1, -0.1, 2.9]
f*: 1.7164262632557858

Iterations: 845
x*: [1.0000000000000007, -0.09, -0.09, 2.920000000000006]
f*: 1.7157256258683309



([1.0000000000000007, -0.09, -0.09, 2.920000000000006], 1.7157256258683309, 845)

In [65]:
x0 = [1.0, 1.0, 1.0, 1.0]
random_hill(levy3, x0; max_failed=100, verbose=true)
random_hill(levy3, x0; h=.01, max_failed=100, verbose=true)

Iterations: 113
x*: [1.0, 1.0, 1.0, 0.8]
f*: 0.536790836378138

Iterations: 195
x*: [1.0, 1.0, 1.0, 0.8399999999999999]
f*: 0.5352769012949443



([1.0, 1.0, 1.0, 0.8399999999999999], 0.5352769012949443, 195)

## 3 Simulated Annealing

In [132]:
max_iter = 1000
SA_options = Optim.Options(iterations=max_iter)

                x_abstol = 0.0
                x_reltol = 0.0
                f_abstol = 0.0
                f_reltol = 0.0
                g_abstol = 1.0e-8
                g_reltol = 1.0e-8
          outer_x_abstol = 0.0
          outer_x_reltol = 0.0
          outer_f_abstol = 0.0
          outer_f_reltol = 0.0
          outer_g_abstol = 1.0e-8
          outer_g_reltol = 1.0e-8
           f_calls_limit = 0
           g_calls_limit = 0
           h_calls_limit = 0
       allow_f_increases = true
 allow_outer_f_increases = true
        successive_f_tol = 1
              iterations = 1000
        outer_iterations = 1000
             store_trace = false
           trace_simplex = false
              show_trace = false
          extended_trace = false
              show_every = 1
                callback = nothing
              time_limit = NaN


In [138]:
x0 = [6.0, -2.0, -2.0, 6.0]
optimize(levy3, x0, SimulatedAnnealing(), SA_options)

 * Status: failure (reached maximum number of iterations)

 * Candidate solution
    Final objective value:     7.632990e-02

 * Found with
    Algorithm:     Simulated Annealing

 * Convergence measures
    |x - x'|               = NaN ≰ 0.0e+00
    |x - x'|/|x'|          = NaN ≰ 0.0e+00
    |f(x) - f(x')|         = NaN ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = NaN ≰ 0.0e+00
    |g(x)|                 = NaN ≰ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    1000
    f(x) calls:    1001


In [134]:
x0 = [4.2, -1.2, 2.4, -3.8]
optimize(levy3, x0, SimulatedAnnealing(), SA_options)

 * Status: failure (reached maximum number of iterations)

 * Candidate solution
    Final objective value:     3.974731e-02

 * Found with
    Algorithm:     Simulated Annealing

 * Convergence measures
    |x - x'|               = NaN ≰ 0.0e+00
    |x - x'|/|x'|          = NaN ≰ 0.0e+00
    |f(x) - f(x')|         = NaN ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = NaN ≰ 0.0e+00
    |g(x)|                 = NaN ≰ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    1000
    f(x) calls:    1001


In [135]:
x0 = [0.0, 0.0, 0.0, 3.2]
optimize(levy3, x0, SimulatedAnnealing(), SA_options)

 * Status: failure (reached maximum number of iterations)

 * Candidate solution
    Final objective value:     5.499646e-02

 * Found with
    Algorithm:     Simulated Annealing

 * Convergence measures
    |x - x'|               = NaN ≰ 0.0e+00
    |x - x'|/|x'|          = NaN ≰ 0.0e+00
    |f(x) - f(x')|         = NaN ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = NaN ≰ 0.0e+00
    |g(x)|                 = NaN ≰ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    1000
    f(x) calls:    1001


In [136]:
x0 = [1.0, 1.0, 1.0, 1.0]
optimize(levy3, x0, SimulatedAnnealing(), SA_options)

 * Status: failure (reached maximum number of iterations)

 * Candidate solution
    Final objective value:     2.699930e-02

 * Found with
    Algorithm:     Simulated Annealing

 * Convergence measures
    |x - x'|               = NaN ≰ 0.0e+00
    |x - x'|/|x'|          = NaN ≰ 0.0e+00
    |f(x) - f(x')|         = NaN ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = NaN ≰ 0.0e+00
    |g(x)|                 = NaN ≰ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    1000
    f(x) calls:    1001


## 4 Particle Swarm

In [116]:
max_iter = 1000
PS_options = Optim.Options(iterations=max_iter)

                x_abstol = 0.0
                x_reltol = 0.0
                f_abstol = 0.0
                f_reltol = 0.0
                g_abstol = 1.0e-8
                g_reltol = 1.0e-8
          outer_x_abstol = 0.0
          outer_x_reltol = 0.0
          outer_f_abstol = 0.0
          outer_f_reltol = 0.0
          outer_g_abstol = 1.0e-8
          outer_g_reltol = 1.0e-8
           f_calls_limit = 0
           g_calls_limit = 0
           h_calls_limit = 0
       allow_f_increases = true
 allow_outer_f_increases = true
        successive_f_tol = 1
              iterations = 1000
        outer_iterations = 1000
             store_trace = false
           trace_simplex = false
              show_trace = false
          extended_trace = false
              show_every = 1
                callback = nothing
              time_limit = NaN


In [117]:
x0 = [6.0, -2.0, -2.0, 6.0]
optimize(levy3, x0, ParticleSwarm(), PS_options)

 * Status: failure (reached maximum number of iterations)

 * Candidate solution
    Final objective value:     3.305329e-11

 * Found with
    Algorithm:     Particle Swarm

 * Convergence measures
    |x - x'|               = NaN ≰ 0.0e+00
    |x - x'|/|x'|          = NaN ≰ 0.0e+00
    |f(x) - f(x')|         = NaN ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = NaN ≰ 0.0e+00
    |g(x)|                 = NaN ≰ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    1000
    f(x) calls:    5004
    ∇f(x) calls:   0


In [119]:
x0 = [4.2, -1.2, 2.4, -3.8]
optimize(levy3, x0, ParticleSwarm(), PS_options)

 * Status: failure (reached maximum number of iterations)

 * Candidate solution
    Final objective value:     8.952825e-02

 * Found with
    Algorithm:     Particle Swarm

 * Convergence measures
    |x - x'|               = NaN ≰ 0.0e+00
    |x - x'|/|x'|          = NaN ≰ 0.0e+00
    |f(x) - f(x')|         = NaN ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = NaN ≰ 0.0e+00
    |g(x)|                 = NaN ≰ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    1000
    f(x) calls:    5004
    ∇f(x) calls:   0


In [120]:
x0 = [0.0, 0.0, 0.0, 3.2]
optimize(levy3, x0, ParticleSwarm(), PS_options)

 * Status: failure (reached maximum number of iterations)

 * Candidate solution
    Final objective value:     1.499760e-32

 * Found with
    Algorithm:     Particle Swarm

 * Convergence measures
    |x - x'|               = NaN ≰ 0.0e+00
    |x - x'|/|x'|          = NaN ≰ 0.0e+00
    |f(x) - f(x')|         = NaN ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = NaN ≰ 0.0e+00
    |g(x)|                 = NaN ≰ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    1000
    f(x) calls:    5004
    ∇f(x) calls:   0


In [121]:
x0 = [1.0, 1.0, 1.0, 1.0]
optimize(levy3, x0, ParticleSwarm(), PS_options)

 * Status: failure (reached maximum number of iterations)

 * Candidate solution
    Final objective value:     5.579929e-23

 * Found with
    Algorithm:     Particle Swarm

 * Convergence measures
    |x - x'|               = NaN ≰ 0.0e+00
    |x - x'|/|x'|          = NaN ≰ 0.0e+00
    |f(x) - f(x')|         = NaN ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = NaN ≰ 0.0e+00
    |g(x)|                 = NaN ≰ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    1000
    f(x) calls:    5004
    ∇f(x) calls:   0


## 5 Analysis